# Rücklaufquote

## Import

In [56]:
import pandas as pd 
import plotly.express as px
pd.set_option('display.max_rows', 500)  # Zeilenanzahl erhöhen
pd.set_option('display.max_columns', 50)  # Spaltenanzahl erhöhen
pd.set_option('display.width', 1000)  # Maximale Breite der Anzeige
import plotly.io as pio
import plotly.graph_objects as go

infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="Titel",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template


In [57]:
years = [str(year) for year in range(2013, 2025)]
df = pd.read_excel("data.xlsx", sheet_name=years)
data_list = []
for year, df in df.items():
    df['Jahr'] = year

    data_list.append(df)
final_df = pd.concat(data_list, ignore_index=True)

In [58]:
final_df = final_df[final_df.columns[~final_df.columns.str.contains(r'\*')]] # drop * in the data
final_df.columns = final_df.columns.str.replace('*', '', regex=False)# just to be sure
#final_df = final_df.rename(columns={'Master Getränketechnologi':'Master Getränketechnologie'})# fix a typo

clean_columns = {}
for col in final_df.columns:
    clean_name = col.strip()  # get rid of other human typos
    if clean_name in clean_columns:
        clean_columns[clean_name].append(col)
    else:
        clean_columns[clean_name] = [col]

In [59]:
df = final_df[final_df['Category']=='Anzahl Studierende'].drop(columns=['Variable', 'Category'])
df_grouped = df.groupby(by='Jahr').sum().reset_index().sort_values(by='Jahr')
df_grouped.sort_values(by='Jahr')
columns_to_plot = df_grouped.columns.to_list()

### Plot

### Heatmap

In [60]:
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="Titel",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template


In [61]:

df = df_grouped

heatmap_data = df.drop("Jahr", axis=1)

# Erstelle die Heatmap mit Plotly Express
fig = px.imshow(heatmap_data, 
                labels=dict(x="Studiengänge", y="Jahr"), 
                x=heatmap_data.columns, 
                y=df["Jahr"],
                color_continuous_scale='viridis')
fig.update_layout(
    title='Beteiligung an der Studierendenumfrage des Fachbereichs 09 nach Jahren',
    width=1400,
    height=700,
    
)
fig.update_layout(
    xaxis=dict(
        title='Studiengänge',
        tickfont=dict(size=11)
    ),
    yaxis=dict(
        title='Jahre',
        tickfont=dict(size=11)
    ),
    coloraxis_colorbar=dict(
        tickfont=dict(size=11),
        len=0.7,


))

fig.show()
fig.write_image("Plots/pdf/ruecklauf1.pdf")

In [62]:
bachelor_columns = [col for col in df.columns if "Bachelor" in col]
master_columns = [col for col in df.columns if "Master" in col]

bachelor_df = df[["Jahr"] + bachelor_columns].melt(id_vars="Jahr", value_name="Anzahl")
bachelor_df["Studiengang"] = "Bachelor"
bachelor_agg = bachelor_df.groupby(["Jahr", "Studiengang"])["Anzahl"].sum().reset_index()

master_df = df[["Jahr"] + master_columns].melt(id_vars="Jahr", value_name="Anzahl")
master_df["Studiengang"] = "Master"
master_agg = master_df.groupby(["Jahr", "Studiengang"])["Anzahl"].sum().reset_index()

final_df = pd.concat([bachelor_agg, master_agg], ignore_index=True).sort_values('Jahr')

In [63]:
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="Entwicklung der Studierendenberfagung im FB09 zwischen 2013 und 2024",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

fig = px.line(final_df, x="Jahr", y="Anzahl", color='Studiengang',title="<b>Beteiligung der Studierendenbefragung in </b><span style='color:#73c6e9;'>Bachelor</span> <b>und </b><span style='color:#cc5b6e;'>Master</span> <b>ist rückläufig </b>", template="infoviz", color_discrete_sequence=px.colors.qualitative.Safe,)
fig.update_layout(
    yaxis=dict(title="Anzahl der Studierenden"),
    showlegend=False,
    height=500,
    width=1500,
    )
fig.show()
fig.write_image("Plots/pdf/ruecklauf2.pdf")

In [64]:
file_path = "data.xlsx"
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")
sheets.pop(next(iter(sheets)))
df_2024 = sheets["2024"]

fragen_anzahl = df_2024[df_2024["Category"] == "Anzahl"]

df_2024[df_2024["Category"] == "Anzahl"]

,Variable,Category,Bachelor Agrarwissenschaften,Bachelor Ernährungswissenschaften,Bachelor Nachwachsende Rohstoffe und Bioressourcen,Bachelor Ökotrophologie,Bachelor Umwelt und globaler Wandel,Master Agrar- und Ressourcenökonomie,Master Agrobiotechnologie,Master Ernährungswissenschaften,Master Getränketechnologie*,Master Informationstechnologie in den Agrar- und Umweltwissenschaften*,Master Insect Biotechnology and Bioresources,Master Nachhaltige Ernährungswirtschaft,Master Nutzpflanzenwissenschaften,Master Nutztierwissenschaften*,Master Oenologie/Weinwirtschaft*,Master Ökotrophologie,Master Sustainable Transition,Master Transition Management,Master Umweltwissenschaften
3,Geschlecht,Anzahl,23,63.0,12.0,44.0,52.0,8.0,27.0,33.0,NaN,NaN,10.0,11.0,3.0,NaN,NaN,22.0,5.0,19.0,17.0
13,Alter,Anzahl,23,63.0,13.0,44.0,53.0,8.0,26.0,33.0,NaN,NaN,9.0,9.0,3.0,NaN,NaN,22.0,5.0,19.0,17.0
17,Staatsangehörigkeit,Anzahl,26,66.0,13.0,45.0,53.0,9.0,28.0,33.0,NaN,NaN,10.0,11.0,3.0,NaN,NaN,22.0,5.0,19.0,17.0
21,Migrationshintergrund,Anzahl,23,62.0,12.0,44.0,53.0,8.0,26.0,33.0,NaN,NaN,10.0,8.0,3.0,NaN,NaN,22.0,4.0,18.0,17.0
37,Fachsemester,Anzahl,34,101.0,18.0,57.0,63.0,13.0,14.0,32.0,NaN,NaN,10.0,12.0,5.0,NaN,NaN,23.0,5.0,13.0,20.0
44,Zufriedenheit,Anzahl,30,88.0,14.0,51.0,60.0,12.0,33.0,36.0,NaN,NaN,12.0,13.0,6.0,NaN,NaN,24.0,6.0,22.0,18.0
49,Subjektive Leistungseinschätzung,Anzahl,28,75.0,14.0,48.0,54.0,10.0,26.0,33.0,NaN,NaN,10.0,12.0,6.0,NaN,NaN,22.0,6.0,20.0,15.0


In [65]:

gesamt_row = df_2024[(df_2024["Variable"] == "Rücklaufquote") & (df_2024["Category"] == "Anzahl Studierende")]
gesamt_studierende = gesamt_row.iloc[:, 2:].sum().sum()

In [66]:
ruecklaufquote_fragen = {"Frage": [], "Rücklaufquote (%)": []}

for _, row in fragen_anzahl.iterrows():
    frage = row["Variable"]
    antworten_anzahl = row.iloc[2:].sum()

    ruecklaufquote = (antworten_anzahl / gesamt_studierende) * 100 if gesamt_studierende > 0 else 0

    ruecklaufquote_fragen["Frage"].append(frage)
    ruecklaufquote_fragen["Rücklaufquote (%)"].append(ruecklaufquote)

df_ruecklauf_fragen = pd.DataFrame(ruecklaufquote_fragen)
df_ruecklauf_fragen = df_ruecklauf_fragen.sort_values(by="Rücklaufquote (%)", ascending=False)

In [ ]:

# Benutzerdefiniertes Template
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="Relative Rücklaufquoten für verschiedene Fragekategorien in der Studierendenbefragung des FB09 in 2024",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

fig = px.bar(
    df_ruecklauf_fragen,
    y="Rücklaufquote (%)",
    x="Frage",
    title="Rücklaufquoten in der Studierendenbefragung: Unterschiede zwischen Themenbereichen ",
    template="infoviz",
    color_discrete_sequence=px.colors.qualitative.Safe
)

fig.update_layout(
    #xaxis=dict(title="Rücklaufquote (%)", range=[0, 20]),
    yaxis=dict(title="Relativer Anteil"),
    height=500,
    width=1500,
    )
fig.show()
fig.write_image("Plots/pdf/ruecklauf3.pdf")

In [68]:
file_path = "data.xlsx"
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")
sheets.pop(next(iter(sheets)))
ruecklaufquote_fragen = {}

for jahr, df in sheets.items():
    fragen_anzahl = df[df["Category"] == "Anzahl"]

    gesamt_row = df[(df["Variable"] == "Rücklaufquote") & (df["Category"] == "Anzahl Studierende")]
    gesamt_studierende = gesamt_row.iloc[:, 2:].sum().sum()

    if gesamt_studierende == 0 or gesamt_row.empty:
        continue

    for _, row in fragen_anzahl.iterrows():
        frage = row["Variable"]
        antworten_anzahl = row.iloc[2:].sum()

        ruecklaufquote = (antworten_anzahl / gesamt_studierende) * 100 if gesamt_studierende > 0 else 0

        if frage not in ruecklaufquote_fragen:
            ruecklaufquote_fragen[frage] = []
        ruecklaufquote_fragen[frage].append(ruecklaufquote)

df_ruecklauf_fragen = pd.DataFrame([
    {"Frage": frage, "Durchschnittliche Rücklaufquote (%)": sum(werte) / len(werte)}
    for frage, werte in ruecklaufquote_fragen.items()
])

df_ruecklauf_fragen = df_ruecklauf_fragen.sort_values(by="Durchschnittliche Rücklaufquote (%)", ascending=False)

fig = px.bar(
    df_ruecklauf_fragen,
    x="Durchschnittliche Rücklaufquote (%)",
    y="Frage",
    orientation="h",
    title="Durchschnittliche Rücklaufquote pro Frage (über alle Jahre)",
    template="infoviz"
)

fig.update_layout(
    xaxis=dict(title="Durchschnittliche Rücklaufquote (%)", range=[0, 20]),
    yaxis=dict(title="Frage"),
    height=800 
)

fig.show()